Nous examinerons toutes les caractéristiques et en sélectionnerons cinq pour créer des clusters. Après avoir construit notre modèle et choisi un nombre approprié de clusters, nous apprendrons à visualiser les clusters multidimensionnelles dans un diagramme de dispersion 2D à l'aide d'une méthode appelée analyse en composantes principales (ACP).

In [2]:
import pandas as pd
import plotly.express as px
from IPython.display import VimeoVideo
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

## Prepare data